# Advent of code 2022 - 07

In [43]:
import re

from utils import read_txt_file

## Code

In [60]:
class FileSystem:
    def __init__(self, debug: bool = False):
        self.debug = debug

        self.current_dir = "/"
        self.parents = {"/": None}
        self.children = {"/": []}
        self.total_file_size = {"/": 0}

    def execute(self, command: str):
        if self.debug:
            print(f"Executing >> '{command}'")

        if command[:2] == "cd":
            self.run_change_directory(command)
        elif command[:2] == "ls":
            self.run_list(command)
        else:
            raise ValueError(f"Wrong command prefix: '{command[:2]}'")

    def run_change_directory(self, command):
        move_to = re.search("(?<=cd )\S+", command).group(0)
        if move_to == "..":
            self.current_dir = self.parents[self.current_dir]
        else:
            self.current_dir = move_to

        if self.debug:
            print(f"-> Changed to new directory: {self.current_dir}")

    def _search_pattern(self, which: str, command: str):
        if which == "dir":
            pattern = re.compile("(?<=dir )\S+")
        elif which == "file":
            pattern = re.compile("^\d+")
        else:
            raise ValueError(f"Wrong value for which: '{which}'")

        try:
            return pattern.search(command).group(0)

        except AttributeError:
            return None

    def _add_dir_relationship(self, parent: str, child: str):
        self.parents[child] = parent

        self.children[parent].append(child)
        self.children[parent] = list(set(self.children[parent]))  # drop duplicates !

        if self.debug:
            print(f"-> parent of '{child}': '{self.parents[child]}'")
            print(f"-> chlidren of '{parent}': '{self.children[parent]}'")

    def _add_total_file_size(self, parent: str, new_file_size: str):
        self.total_file_size[parent] += int(new_file_size)

    def run_list(self, command):
        # Initalisation
        self.children = {self.current_dir: []}
        self.total_file_size[self.current_dir] = 0

        # Data capture
        for f in command.split("\n")[1:]:
            dir_pattern = self._search_pattern(which="dir", command=f)
            file_pattern = self._search_pattern(which="file", command=f)
            # print(f"{f}: dir pattern found: '{dir_pattern}'")
            # print(f"{f}: file pattern found: '{file_pattern}'")
            if dir_pattern:
                self._add_dir_relationship(parent=self.current_dir, child=dir_pattern)
            elif file_pattern:
                self._add_total_file_size(
                    parent=self.current_dir, new_file_size=file_pattern
                )
            else:
                raise ValueError(f"No pattern found in '{f}'")

        if self.debug:
            print(
                f"-> total file size of '{self.current_dir}': '{self.total_file_size[self.current_dir]}'"
            )


def solve_07(path: str, debug: bool = False):
    input_txt = read_txt_file(path)

    fs = FileSystem(debug=debug)

    commands = [x.replace("$ ", "") for x in input_txt.split("\n$ ")]
    for i, c in enumerate(commands):
        fs.execute(c)
        if debug:
            print()

    return "TODO"


# Run some tests
example_file = f"inputs/07_example.txt"
solve_07(example_file, debug=True)

Executing >> 'cd /'
-> Changed to new directory: /

Executing >> 'ls
dir a
14848514 b.txt
8504156 c.dat
dir d'
-> parent of 'a': '/'
-> chlidren of '/': '['a']'
-> parent of 'd': '/'
-> chlidren of '/': '['d', 'a']'
-> total file size of '/': '23352670'

Executing >> 'cd a'
-> Changed to new directory: a

Executing >> 'ls
dir e
29116 f
2557 g
62596 h.lst'
-> parent of 'e': 'a'
-> chlidren of 'a': '['e']'
-> total file size of 'a': '94269'

Executing >> 'cd e'
-> Changed to new directory: e

Executing >> 'ls
584 i'
-> total file size of 'e': '584'

Executing >> 'cd ..'
-> Changed to new directory: a

Executing >> 'cd ..'
-> Changed to new directory: /

Executing >> 'cd d'
-> Changed to new directory: d

Executing >> 'ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k'
-> total file size of 'd': '24933642'



'TODO'